<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=3362d64505a7d474e3f4c5abba3078f8ace2e7cd3c11bb8976bfe54b5749b7cc
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-17 14:28:56
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  3.00 K
Current:  1.46 C
-------------------
Today PnL: 40.60 K (0.28%)
Current PnL: -19.80 L (-12.98%)
CY Booked + Current PnL: -6.28 L (-4.12%)
-------------------
Total profit:  2.15 L
Total loss:  -21.95 L
-------------------
Total Booked + Current PnL: 20.70 L (16.48%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.28%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.36 C
Est FTT PnL: 90.68 L (62.27%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.54%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,147.77,84.0,M-SC,0.70,162366.0,-12193.0,12226.0,0.82,-6.98,7.53,0.02,234.0,-1.00,1.12,41.23,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,101.92,51.0,M-SC,3.42,86971.0,-13806.0,13898.0,0.05,-13.70,15.98,0.09,253.0,-0.99,0.60,14.26,OX40N,NTT,DURABLES
5,ANGELONE,3033.00,66.50,79.0,X-SC,7.44,234094.0,43088.0,14607.0,3.99,22.56,6.24,30.21,97.0,2.95,1.61,48.15,X40N,NTT,FINANCE
2,ABBOTINDIA,35195.00,-15.14,47.0,X-MC,3.57,88530.0,-2148.0,17051.0,0.58,-2.37,19.26,16.44,63.0,-0.13,0.61,17.33,X40,ATH,PHARMA
35,ICICIGI,2252.93,-15.07,58.0,X-MC,7.51,188985.0,14665.0,20543.0,1.35,8.41,10.87,20.19,68.0,0.71,1.30,24.80,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SURYODAY,240.0,64.47,54.0,H-SC,13.67,151113.0,-27958.0,93690.0,7.72,-15.61,62.00,36.71,167.0,-0.30,1.04,49.93,XR,NTT,BANKS
62,SAMMAANCAP,326.0,22.90,62.0,M-SC,37.11,165816.0,16596.0,127579.0,4.60,11.12,76.94,96.62,245.0,0.13,1.14,79.45,XY25,NTT,FINANCE
5,ANGELONE,3033.0,66.50,79.0,X-SC,7.44,234094.0,43088.0,14607.0,3.99,22.56,6.24,30.21,97.0,2.95,1.61,48.15,X40N,NTT,FINANCE
66,SIEMENS,4671.5,-1.82,60.0,H-LC,1.55,160130.0,-25965.0,73452.0,3.84,-13.95,45.87,25.51,49.0,-0.35,1.10,18.00,AR,ATH,ELECTRICAL
0,5PAISA,593.0,81.49,31.0,H-SC,17.04,123427.0,-95069.0,123267.0,2.61,-43.51,99.87,12.90,161.0,-0.77,0.85,2.63,OX40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.0,-20.63,25.0,X-LC,0.27,243922.0,-32488.0,453646.0,-4.81,-11.75,185.98,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
19,COFFEEDAY,80.0,-51.71,32.0,L-SC,21.17,70963.0,-42586.0,82636.0,-3.75,-37.50,116.45,35.27,270.0,-0.52,0.49,73.68,XR,NTT,HOTELS
24,EASEMYTRIP,26.4,-21.71,33.0,M-SC,20.26,75332.0,-101483.0,182281.0,-2.89,-57.40,241.97,45.70,205.0,-0.56,0.52,0.00,XY24,NTT,TRAVEL
64,SFL,1287.0,24.89,35.0,M-SC,3.06,164849.0,-97369.0,167190.0,-2.83,-37.13,101.42,26.63,239.0,-0.58,1.13,2.49,XY24,NTT,MISC
28,GREENPANEL,537.0,192.64,35.0,M-SC,5.39,130784.0,-52294.0,131268.0,-1.47,-28.56,100.37,43.14,240.0,-0.40,0.90,23.02,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-41.83,39.0,M-SC,8.89,113988.0,-810.0,94849.0,-0.48,-0.71,83.21,81.92,236.0,-0.01,0.78,34.94,XR,NTT,DURABLES
25,FINCABLES,1641.55,-18.12,51.0,M-SC,10.19,148665.0,1021.0,158298.0,1.53,0.69,106.48,107.91,216.0,0.01,1.02,3.71,OX40N,ATH,CABLES
69,STARHEALTH,761.00,37.72,71.0,H-SC,15.52,283832.0,2709.0,130165.0,-0.13,0.96,45.86,47.26,171.0,0.02,1.95,52.07,XY24,NTT,INSURANCE
85,WIPRO,420.00,-14.25,52.0,M-LC,5.58,151156.0,211.0,109240.0,-0.23,0.14,72.27,72.51,99.0,0.00,1.04,6.17,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.12,51.0,M-SC,10.19,148665.0,1021.0,158298.0,1.53,0.69,106.48,107.91,216.0,0.01,1.02,3.71,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,147.77,84.0,M-SC,0.70,162366.0,-12193.0,12226.0,0.82,-6.98,7.53,0.02,234.0,-1.00,1.12,41.23,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,101.92,51.0,M-SC,3.42,86971.0,-13806.0,13898.0,0.05,-13.70,15.98,0.09,253.0,-0.99,0.60,14.26,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-65.98,48.0,H-SC,2.36,222696.0,-46971.0,83311.0,-0.55,-17.42,37.41,13.47,158.0,-0.56,1.53,13.23,XY24,NTT,PAINTS
67,SIS,528.00,2019.69,47.0,H-SC,2.37,85540.0,-25492.0,49100.0,-0.22,-22.96,57.40,21.26,166.0,-0.52,0.59,15.57,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,3033.0,66.5,79.0,X-SC,7.44,234094.0,43088.0,14607.0,3.99,22.56,6.24,30.21,97.0,2.95,1.61,48.15,X40N,NTT,FINANCE


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,37.72,71.0,H-SC,15.52,283832.0,2709.0,130165.0,-0.13,0.96,45.86,47.26,171.0,0.02,1.95,52.07,XY24,NTT,INSURANCE
62,SAMMAANCAP,326.00,22.90,62.0,M-SC,37.11,165816.0,16596.0,127579.0,4.60,11.12,76.94,96.62,245.0,0.13,1.14,79.45,XY25,NTT,FINANCE
85,WIPRO,420.00,-14.25,52.0,M-LC,5.58,151156.0,211.0,109240.0,-0.23,0.14,72.27,72.51,99.0,0.00,1.04,6.17,XR,NTT,IT
25,FINCABLES,1641.55,-18.12,51.0,M-SC,10.19,148665.0,1021.0,158298.0,1.53,0.69,106.48,107.91,216.0,0.01,1.02,3.71,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-47.39,55.0,H-SC,4.44,131228.0,7892.0,123735.0,1.00,6.40,94.29,106.72,139.0,0.06,0.90,31.27,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.39,55.0,H-SC,4.44,131228.0,7892.0,123735.0,1.00,6.40,94.29,106.72,139.0,0.06,0.90,31.27,AR,ATH,MISC
25,FINCABLES,1641.55,-18.12,51.0,M-SC,10.19,148665.0,1021.0,158298.0,1.53,0.69,106.48,107.91,216.0,0.01,1.02,3.71,OX40N,ATH,CABLES
85,WIPRO,420.00,-14.25,52.0,M-LC,5.58,151156.0,211.0,109240.0,-0.23,0.14,72.27,72.51,99.0,0.00,1.04,6.17,XR,NTT,IT
84,WHIRLPOOL,2270.00,-41.83,39.0,M-SC,8.89,113988.0,-810.0,94849.0,-0.48,-0.71,83.21,81.92,236.0,-0.01,0.78,34.94,XR,NTT,DURABLES
86,ZYDUSLIFE,1286.17,-15.41,34.0,H-MC,5.59,206480.0,-3094.0,77760.0,-1.07,-1.48,37.66,35.63,119.0,-0.04,1.42,14.95,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,10.47,24.0,X-SC,9.78,84278.0,-44392.0,89689.0,-1.14,-34.50,106.42,35.20,92.0,-0.49,0.58,0.00,X40,NTT,FOOTWEAR
76,TMPV,1065.00,-20.63,25.0,X-LC,0.27,243922.0,-32488.0,453646.0,-4.81,-11.75,185.98,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
27,GLAXO,3466.20,-19.65,30.0,X-MC,6.40,199280.0,-4.0,78018.0,0.47,-0.00,39.15,39.15,60.0,-0.00,1.37,29.00,X40,ATH,PHARMA
23,DMART,5391.45,-17.28,31.0,X-LC,5.29,201800.0,-8607.0,67764.0,-0.44,-4.09,33.58,28.12,19.0,-0.13,1.39,18.57,X40N,ATH,FMCG
46,JSWINFRA,342.00,-19.06,31.0,X-MC,5.67,189540.0,-10807.0,41301.0,-0.04,-5.39,21.79,15.23,66.0,-0.26,1.30,25.12,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.00,-20.63,25.0,X-LC,0.27,243922.0,-32488.0,453646.0,-4.81,-11.75,185.98,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
43,ITC,452.00,-40.74,46.0,X-LC,0.91,198974.0,-1164.0,22046.0,-0.31,-0.58,11.08,10.44,5.0,-0.05,1.37,4.95,X40,NTT,FMCG
34,HONAUT,58357.33,-25.23,35.0,X-SC,1.01,106275.0,-21663.0,68792.0,-0.30,-16.93,64.73,36.84,90.0,-0.31,0.73,8.80,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-20.00,67.0,X-MC,1.19,184780.0,8042.0,46694.0,0.19,4.55,25.27,30.97,75.0,0.17,1.27,17.68,X40,ATH,INSURANCE
66,SIEMENS,4671.50,-1.82,60.0,H-LC,1.55,160130.0,-25965.0,73452.0,3.84,-13.95,45.87,25.51,49.0,-0.35,1.10,18.00,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,10.47,24.0,X-SC,9.78,84278.0,-44392.0,89689.0,-1.14,-34.50,106.42,35.20,92.0,-0.49,0.58,0.00,X40,NTT,FOOTWEAR
76,TMPV,1065.00,-20.63,25.0,X-LC,0.27,243922.0,-32488.0,453646.0,-4.81,-11.75,185.98,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
53,PAGEIND,51769.93,-28.95,36.0,X-MC,7.78,157940.0,-5800.0,49135.0,-0.70,-3.54,31.11,26.47,55.0,-0.12,1.08,0.07,X40,ATH,APPARELS
10,BAJAJHFL,181.50,-18.37,36.0,X-MC,6.62,176136.0,-24370.0,121516.0,-0.13,-12.15,68.99,48.45,64.0,-0.20,1.21,1.57,X40N,ATH,FINANCE
55,QUESS,986.00,-52.62,33.0,X-SC,37.21,47304.0,-17702.0,168629.0,1.96,-27.23,356.48,232.18,83.0,-0.10,0.32,1.96,XY24,NTT,MISC


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,986.00,-52.62,33.0,X-SC,37.21,47304.0,-17702.0,168629.0,1.96,-27.23,356.48,232.18,83.0,-0.10,0.32,1.96,XY24,NTT,MISC
59,RELAXO,1176.00,-41.05,41.0,X-SC,3.24,80332.0,-64828.0,140758.0,0.27,-44.66,175.22,52.31,91.0,-0.46,0.55,7.53,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,10.47,24.0,X-SC,9.78,84278.0,-44392.0,89689.0,-1.14,-34.50,106.42,35.20,92.0,-0.49,0.58,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-15.14,47.0,X-MC,3.57,88530.0,-2148.0,17051.0,0.58,-2.37,19.26,16.44,63.0,-0.13,0.61,17.33,X40,ATH,PHARMA
34,HONAUT,58357.33,-25.23,35.0,X-SC,1.01,106275.0,-21663.0,68792.0,-0.30,-16.93,64.73,36.84,90.0,-0.31,0.73,8.80,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-26.54,59.0,X-LC,8.48,291889.0,-54067.0,120754.0,-0.03,-15.63,41.37,19.28,1.0,-0.45,2.00,7.91,X40,ATH,IT
41,INFY,2275.00,-19.96,54.0,X-LC,4.06,320906.0,8018.0,163662.0,0.25,2.56,51.00,54.87,2.0,0.05,2.20,11.06,X40,BTT,IT
76,TMPV,1065.00,-20.63,25.0,X-LC,0.27,243922.0,-32488.0,453646.0,-4.81,-11.75,185.98,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
82,VBL,671.64,-17.80,49.0,X-LC,5.98,294628.0,-21214.0,133879.0,0.52,-6.72,45.44,35.67,4.0,-0.16,2.02,6.21,X40N,ATH,FMCG
43,ITC,452.00,-40.74,46.0,X-LC,0.91,198974.0,-1164.0,22046.0,-0.31,-0.58,11.08,10.44,5.0,-0.05,1.37,4.95,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-51.71,32.0,L-SC,21.17,70963.0,-42586.0,82636.0,-3.75,-37.50,116.45,35.27,270.0,-0.52,0.49,73.68,XR,NTT,HOTELS
7,ASIANTILES,137.00,7616.67,65.0,L-SC,17.13,84692.0,-9118.0,85734.0,-0.12,-9.72,101.23,81.67,271.0,-0.11,0.58,64.01,XR,NTT,CERAMICS
51,MASFIN,398.61,-17.80,54.0,H-SC,7.50,93510.0,-4470.0,26071.0,0.79,-4.56,27.88,22.05,164.0,-0.17,0.64,35.92,XR,ATH,FINANCE
38,INDIAMART,4810.62,-47.39,55.0,H-SC,4.44,131228.0,7892.0,123735.0,1.00,6.40,94.29,106.72,139.0,0.06,0.90,31.27,AR,ATH,MISC
80,VAIBHAVGBL,521.00,72.45,61.0,H-SC,9.44,149545.0,-33230.0,144296.0,0.02,-18.18,96.49,60.77,159.0,-0.23,1.03,34.68,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,147.77,84.0,M-SC,0.70,162366.0,-12193.0,12226.0,0.82,-6.98,7.53,0.02,234.0,-1.00,1.12,41.23,OX40N,NTT,PAINTS
80,VAIBHAVGBL,521.00,72.45,61.0,H-SC,9.44,149545.0,-33230.0,144296.0,0.02,-18.18,96.49,60.77,159.0,-0.23,1.03,34.68,XR,NTT,JEWELLERY
6,ASIANPAINT,3465.66,-0.07,83.0,X-LC,9.51,254276.0,2508.0,50703.0,-0.58,1.00,19.94,21.13,31.0,0.05,1.75,36.77,X40,ATH,PAINTS
5,ANGELONE,3033.00,66.50,79.0,X-SC,7.44,234094.0,43088.0,14607.0,3.99,22.56,6.24,30.21,97.0,2.95,1.61,48.15,X40N,NTT,FINANCE
14,BSOFT,831.70,-9.93,61.0,H-SC,8.60,98596.0,-42057.0,113484.0,-0.53,-29.90,115.10,50.79,151.0,-0.37,0.68,14.80,XR,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.58
1,25,44.48
2,50,75.72


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    47.68
MC    27.74
LC    24.61
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.29
X40      20.51
X40N     12.30
XY25     10.12
XR        9.60
AR        9.01
OX40N     7.58
X200      1.80
SR        1.01
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.31
X-MC    21.37
X-LC    19.68
M-SC    13.32
X-SC     8.63
H-MC     4.67
H-LC     2.90
L-SC     1.42
M-MC     1.37
M-LC     1.04
L-LC     0.99
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.63,-4.01,37.64
IT,12.46,-15.79,77.70
FINANCE,12.04,-7.91,56.83
MISC,7.48,-19.22,79.35
PAINTS,6.00,-5.86,21.64
ELECTRICAL,5.62,-10.38,49.51
INSURANCE,4.52,3.86,30.02
PHARMA,3.40,-1.06,34.97
AUTO,3.36,-17.50,130.14


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3334606.0,20
AR,1259051.0,10
XR,1257153.0,13
X40,984622.0,14
OX40N,689856.0,10
X40N,677456.0,9
XY25,462578.0,7
SR,274264.0,2
MH,66764.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3466896.0,25
M-SC,1412218.0,16
X-LC,1203155.0,11
X-MC,1131162.0,15
X-SC,820097.0,8
H-MC,398763.0,3
L-SC,255136.0,3
H-LC,134905.0,2
M-LC,109240.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1165132.0      6
           AR         843393.0      5
           XR         782868.0      7
M-SC       XY24       748746.0      6
X-LC       XY24       518150.0      2
X-MC       X40        408343.0      7
X-LC       X40        407480.0      5
X-SC       XY24       374672.0      2
X-MC       XY24       346913.0      3
H-SC       OX40N      334475.0      4
X-SC       X40N       276626.0      4
H-SC       SR         274264.0      2
M-SC       OX40N      268615.0      5
H-MC       AR         217770.0      2
X-LC       X40N       201643.0      2
X-MC       X40N       199187.0      3
H-MC       XY24       180993.0      1
X-MC       XY25       176719.0      2
X-SC       X40        168799.0      2
L-SC       XR         168370.0      2
M-SC       XR         142842.0      2
           XY25       127579.0      1
           AR         124436.0      2
M-LC       XR         109240.0      1
L-SC       OX40N       86766.0      1
X-LC       XY25        75882.0      2
H-LC       AR          73452.0      1
H-SC       MH          66764.0      1
H-LC       X200        61453.0      1
L-MC       XR          53833.0      1
M-MC       XY25        46565.0      1
L-LC       XY25        35833.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
